# Datamining
## Assignment 2 - Exercise 2.1
<br></br>
<font size="4">
    This exercise will be split into 2 notebooks in order to save time on each re-run.
    

In [1]:
import pandas as pd

Read .json files as a file stream and keep only useful info (toronto)

In [2]:
businesses = []
with open('yelp_academic_dataset_business.json',encoding="utf8") as f:
    for line in f:
        c = line.find("\"city\"")
        cEnd = line.find("\"state\"")
        c = line[c+2+len("\"city\""):cEnd-2].replace(" ","")
        if(c == "Toronto"):
            #line = line.split(",")
            bizID = line.split("\"business_id\"")[1].split(",")[0][2:-1]
            bizReviewCount = line.split("\"review_count\"")[1].split(",")[0][1:]
            if(int(bizReviewCount) >= 15):
                bizDataToKeep = [bizID,bizReviewCount]
                businesses.append(bizDataToKeep)
f.close()

Create a dataframe for the businesses

In [3]:
df = pd.DataFrame(businesses,columns=["business_id","review_count"])
bizIDList = list(df.business_id)
df
torontoBiz = {}
for i in bizIDList:
    torontoBiz[i] = 1
del bizIDList
df

,business_id,review_count
0,EosRKXIGeSWFYWwpkbhNnA,16
1,0QjROMVW9ACKjhSEfHqNCQ,116
2,GtQPbazps0jt7_cJghfmsA,41
3,xSveVu4UWuyayq8HFlf9KA,21
4,NLaK58WvlNQdUunSIkt-jA,24
...,...,...
7597,yFQCdWr_k1pTObzHPGis9Q,177
7598,0hudPyuCBlKg79OwKBw-eQ,153
7599,GAgEoHcf4PSuZRS5Zd3ltA,16
7600,BtTCTHwoIfr3fBQjaOkpIw,22


Get only toronto reviews.

In [4]:
userDict = {}
userRatings = []
with open('yelp_academic_dataset_review.json',encoding="utf8") as f:
    count = 0
    lines = f.readlines(1)

    while lines:
        line = lines[0]
        bizID = line.split("\"business_id\"")[1].split(",")[0][2:-1]
        if(torontoBiz.get(bizID) == 1):
            userID = line.split("\"user_id\"")[1].split(",")[0][2:-1]
            rating = line.split("\"stars\"")[1].split(",")[0][1:]
            userbiz = ((userID,bizID),rating)
            userRatings.append(userbiz)
            #count times user has rated a toronto biz in order to remove those with <15 later.
            if(userDict.get(userID) == None):
                userDict[userID] = 1
            else:
                userDict[userID] = userDict.get(userID) + 1
        lines = f.readlines(1)

Keep only users with 15+ reviews.

In [5]:
for i in userDict.copy():
    if(userDict.get(i) < 15):
        del userDict[i]
print(len(userDict))

5874


In [6]:
userRatingsO15 = []
for i in range(len(userRatings)):
    if(userDict.get(userRatings[i][0][0]) != None):
        userRatingsO15.append(userRatings[i])
del userRatings

Remake reviews into a dictionary as dictionaries have only unique keys. We use this as it can easily help us keep only unique (user_id,biz_id) key thus keeping only one unique business review per user. 

In [7]:
review15Toronto = dict(userRatingsO15)
len(review15Toronto)

242227

In [8]:
review15Toronto.get(('fwJJjE2HYubWzJia4LbpPQ', '8XVAcOZ02IGTQE-pvKhXXw'))

'5.0'

### In the end we have a dictionary storing as a key a user who reviews the business i.e: (user_id,biz_id), and a value of user_rating.